## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.8961,-24.4956,73.42,77,4,12.75,clear sky
1,1,Kantilo,IN,20.3500,85.1833,67.86,88,4,1.66,clear sky
2,2,San Cristobal,VE,7.7669,-72.2250,68.31,56,19,1.97,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,78.69,73,100,8.05,overcast clouds
4,4,Paamiut,GL,61.9940,-49.6678,12.67,60,100,20.22,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.8961,-24.4956,73.42,77,4,12.75,clear sky
3,3,Rikitea,PF,-23.1203,-134.9692,78.69,73,100,8.05,overcast clouds
12,12,Busselton,AU,-33.6500,115.3333,71.04,39,96,16.20,overcast clouds
15,15,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,94,75,5.75,broken clouds
17,17,Vaini,TO,-21.2000,-175.2000,86.16,74,20,6.91,few clouds
19,19,Buta,CD,2.7858,24.7300,72.27,59,89,0.81,overcast clouds
23,23,Yarada,IN,17.6500,83.2667,70.12,100,20,2.30,mist
30,30,Hilo,US,19.7297,-155.0900,74.80,86,29,5.99,scattered clouds
31,31,Port Alfred,ZA,-33.5906,26.8910,73.63,95,100,9.84,moderate rain
32,32,Ariquemes,BR,-9.9133,-63.0408,73.31,97,83,0.43,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                260
City                   260
Country                258
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                258
City                   258
Country                258
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Current Description    258
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,73.42,clear sky,14.8961,-24.4956,
3,Rikitea,PF,78.69,overcast clouds,-23.1203,-134.9692,
12,Busselton,AU,71.04,overcast clouds,-33.6500,115.3333,
15,Arraial Do Cabo,BR,75.16,broken clouds,-22.9661,-42.0278,
17,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,
19,Buta,CD,72.27,overcast clouds,2.7858,24.7300,
23,Yarada,IN,70.12,mist,17.6500,83.2667,
30,Hilo,US,74.80,scattered clouds,19.7297,-155.0900,
31,Port Alfred,ZA,73.63,moderate rain,-33.5906,26.8910,
32,Ariquemes,BR,73.31,broken clouds,-9.9133,-63.0408,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
     # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
       
             

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.dropna()
hotel_df.count()

#Review that CSV file has proper format
hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,73.42,clear sky,14.8961,-24.4956,Hotel Xaguate
3,Rikitea,PF,78.69,overcast clouds,-23.1203,-134.9692,People ThankYou
12,Busselton,AU,71.04,overcast clouds,-33.6500,115.3333,Observatory Guest House
15,Arraial Do Cabo,BR,75.16,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
17,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [10]:
# 8a. Create the output File (CSV)

clean_hotel_df=hotel_df.copy()
output_data_file = "/Users/ramonmanrique/Desktop/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd> and <dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))